<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 01 - SQL Server Master Instance Queries

In this tutorial you will learn how to run standard SQL Server Queries against the Master Instance (MI) in a SQL Server big data cluster. 

We'll start with a simple set of queries to explore the Instance: 



In [15]:
/* Instance Version */
SELECT @@VERSION; 
GO

/* General Configuration */
USE master;  
GO  
EXEC sp_configure;
GO

/* Databases on this Instance */
SELECT db.name AS 'Database Name'
, Physical_Name AS 'Location on Disk'
, Cast(Cast(Round(cast(mf.size as decimal) * 8.0/1024000.0,2) as decimal(18,2)) as nvarchar) 'Size (GB)'
FROM sys.master_files mf
INNER JOIN 
    sys.databases db ON db.database_id = mf.database_id
WHERE mf.type_desc = 'ROWS';
GO

SELECT * from sys.master_files


## Ingest data into the SQL Server Databases

Before we start working with data, we need to bring it in to the system. We have several options to do that, from the `bcp` utility to SQL Server Integration Services, the Azure Data Factory and more. 

For the structured data, we'll use the SQL Server `RESTORE` command to bring in databases from a location on the web, where we have database backups stored in a Blob (Binary Large Object) inside Microsoft Azure. To restore databases from that blob, we'll use a kind of authentication called a "Shared Access Signature" - a sort of token that allows you to access a file for a short period of time. We'll first create that Token as a SQL Server "Credential". 

We'll then use the `RESTORE` command in SQL Server to restore the databases from the URL, which will call out to the Credential automatically. 

The Code below shows all of that: 

In [3]:
USE master
/* 
DROP CREDENTIAL [https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups]
GO
*/

CREATE CREDENTIAL [https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups] 
   WITH IDENTITY = 'SHARED ACCESS SIGNATURE',  
   SECRET = 'sp=r&st=2019-03-28T18:31:35Z&se=2020-03-29T02:31:35Z&spr=https&sv=2018-03-28&sig=psWVkJ%2F41JMmTKziszSduVYDGOYL546GmaaEsbowi6Q%3D&sr=b';

RESTORE DATABASE [WideWorldImportersDW] 
    FROM URL = 'https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/WWIDW.bak'
GO

## Query Data

The SQL Server 2019 big data cluster Master Instance is a SQL Server Instance - and as such has most all of the query facilities and capabilities of Microsoft SQL Server running on Linux.

**TODO:** Run some standard queries. Investigate simple ML.